In [262]:
import os
import matplotlib.pyplot as plt 
import tensorflow as tf
import numpy as np

from tensorflow.keras import layers 
from tensorflow.keras import Model
from tensorflow import keras

from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.optimizers import RMSprop
from keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten 
from tensorflow.keras.layers import Dense
from keras.utils.np_utils import to_categorical

In [263]:
base_dir= '/Users/Smit/jupyterfiles/skincancer'

train_dir=os.path.join(base_dir, 'train')
test_dir=os.path.join (base_dir, 'test')

benign_train_dir= os.path.join(train_dir,'benign')
malignant_train_dir= os.path.join(train_dir,'malignant') 

benign_val_dir = os.path.join(test_dir, 'benign') 
malignant_val_dir = os.path.join(test_dir, 'malignant')

In [264]:
num_benign_train_dir= len(os.listdir(benign_train_dir))
num_malignant_train_dir= len(os.listdir(malignant_train_dir))

num_benign_val_dir= len(os.listdir(benign_val_dir))
num_malignant_val_dir = len(os.listdir(malignant_val_dir))

total_train = num_benign_train_dir + num_malignant_train_dir 
total_val= num_benign_val_dir + num_malignant_val_dir  

print('total training Benign keratosis images:', num_benign_train_dir) 
print('total training melanoma images:', num_malignant_train_dir)
 
print('total val Benign keratosis images:', num_benign_val_dir)
print('total val melonama images:', num_malignant_val_dir) 

print("---")
print("Total training images:", total_train)
print("Total validation images:", total_val)

total training Benign keratosis images: 115
total training melanoma images: 115
total val Benign keratosis images: 25
total val melonama images: 25
---
Total training images: 230
Total validation images: 50


In [265]:
train_image_generator=ImageDataGenerator(rescale=1./255,
                                         rotation_range=45,
                                         width_shift_range=.15,
                                         height_shift_range=.15,
                                         zoom_range=0.5,
                                         horizontal_flip= True,
                                         vertical_flip= True)
print(train_image_generator)

In [266]:
validation_image_generator = ImageDataGenerator(rescale=1./255)

In [267]:
train_data_gen=train_image_generator.flow_from_directory(batch_size=3,
                                                         directory=train_dir,
                                                         shuffle= True,
                                                         target_size=(150,150),
                                                         class_mode='binary',
                                                         color_mode='rgb')
val_data_gen=validation_image_generator.flow_from_directory(batch_size=3,
                                                         directory=test_dir,
                                                         target_size=(150,150),
                                                         class_mode='binary',
                                                         color_mode='rgb')

Found 230 images belonging to 2 classes.
Found 50 images belonging to 2 classes.


In [268]:
# def classLabel(source=None):
    
#     datagen=ImageDataGenerator (rescale=1. / 255)
#     data_generator =datagen.flow_from_directory(source)
#     class_dictionary =data_generator.class_indices
#     return data_generator, class_dictionary

# labels = classLabel(train_dir)
# print (labels)

In [ ]:
train_data_gen = to_categorical(train_data_gen, num_classes= 2)
val_data_gen = to_categorical(val_data_gen, num_classes= 2)

In [ ]:
sample_training_images, _ = next(train_data_gen)

def plotImages(images_arr):
    
    fig, axes = plt.subplots (1, 2, figsize=(20,20)) 
    for img, ax in zip( images_arr, axes): 
        ax.imshow(img)

plt.tight_layout()
plt.show()
plotImages(sample_training_images[:2])

In [ ]:
model = Sequential()
model.add(Conv2D(64, 3, padding='same', activation='relu', input_shape=(150,150,3,)))
model.add (MaxPooling2D (pool_size=2))
model.add (Dropout(0.25))

model.add (Conv2D(64, 3, padding='same', activation='relu'))
model.add (MaxPooling2D (pool_size=2))
model.add(Dropout (0.25))

model.add(Flatten())
model.add (Dense (128, activation='relu'))

model.add (Dense(2, activation='softmax'))
model.compile(optimizer = 'adam',
               loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
print(model.summary())

In [ ]:
history = model.fit(
                  train_data_gen,
                  validation_data= val_data_gen,
validation_split=0.2,
                  steps_per_epoch=len(train_data_gen),
                  epochs=20,
                  validation_steps = len(val_data_gen),
                  verbose = 2)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history [ 'val_accuracy'] 
val_loss= history.history['val_loss']
loss= history.history['loss']

epochs = range (len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='validation accuracy')
plt.title('Training and validation accuracy')    
plt.legend(loc=8)
plt.figure()

plt.show()

In [ ]:
Y_pred = model.predict(val_data_gen,train_data_gen)
y_pred = np.argmax(Y_pred, axis=1)

In [ ]:
acc= model.evaluate_generator(val_data_gen) [1]
print (f"the accuracy of our model is {acc*100} %")

In [ ]:
from sklearn.metrics import confusion_matrix
print('Confusion Matrix')
print(confusion_matrix(val_data_gen.classes, y_pred))

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

img_path = "/Users/Smit/jupyterfiles/skincancer/test/malignant/1401.jpg"
img = image.load_img(img_path, target_size=(150, 150))
plt.imshow(img)
plt.show()

In [ ]:
import numpy as np
img_array = image.img_to_array(img)
img_batch = np.expand_dims(img_array, axis=0)
img_preprocessed = preprocess_input(img_batch)
prediction = model.predict(img_batch)
classes=  np.argmax(prediction)


    
if (classes == 0):
    prediction = "benign"
    print("No cancer")
else:
    prediction = "malignant"
    print("Yes cancer")

        

In [ ]:
model.save("model.h5")